In [11]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.9/966.9 kB 65.1 MB/s eta 0:00:00


In [12]:
import pandas as pd
from underthesea import word_tokenize 
import string
from gensim.utils import simple_preprocess
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

In [13]:
df = pd.read_csv("../input/btmidtermnlp/data.csv")
df.head(5)

,title,category,content
0,Galaxy Gear có thể điều khiển TV thông minh,Công nghệ,Mẫu đồng hồ thông minh Galaxy Gear của Samsung...
1,Nhật – Mỹ định hội đàm chớp nhoáng tại Singapo...,Thế giới,">> Phó Tổng thống Mỹ công du châu Á, bàn về Bi..."
2,Hàng ngàn học sinh miền núi Nghệ An bỏ học,Giáo dục,Hàng ngàn học sinh sau khi tốt nghiệp THCS đã ...
3,'Cô ấy bỏ em vì... em gái trượt đại học?',Đời sống,Mr Búp Bê (BB): Hay là bạn phải đến kèm cho cô...
4,Cậu bé vượt qua Face ID của iPhone ngang anh Q...,Công nghệ,Trang Wired mới đây đưa thông tin cậu bé Ammar...


In [14]:
df['content'].head(5)

0    Mẫu đồng hồ thông minh Galaxy Gear của Samsung...
1    >> Phó Tổng thống Mỹ công du châu Á, bàn về Bi...
2    Hàng ngàn học sinh sau khi tốt nghiệp THCS đã ...
3    Mr Búp Bê (BB): Hay là bạn phải đến kèm cho cô...
4    Trang Wired mới đây đưa thông tin cậu bé Ammar...
Name: content, dtype: object

In [15]:
data = df['content']

In [16]:
def remove_stop_words(sentence,stopword):
    results = []
    tmp = sentence.split(' ')
    for stop_word in stopword:
        if stop_word in tmp:
            tmp.remove(stop_word)
    results.append(" ".join(tmp))
    return results

In [17]:
def get_stopwords_list(stop_file_path):
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [18]:
stop_word = get_stopwords_list("/kaggle/input/vietnamesestopword/vietnamese-stopwords.txt")

In [19]:
len(stop_word)

1941

In [20]:
def preprocessingSentence(sentence):
    # !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
    for character in string.punctuation:
        sentence = sentence.replace(character,'')
    sentence = ' '.join(simple_preprocess(sentence))
    sentence = word_tokenize(sentence, format='text').lower()
    content = remove_stop_words(sentence ,stop_word)
    return content

In [21]:
df = list()
for i in data:
    df.append(i)

In [22]:
resultSentence = []
for i in df:
    i = preprocessingSentence(i)
    resultSentence.append(i)

In [67]:
def save_Content(ListContent, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in ListContent:
            f.write("%s\n" % item)
        f.close()

def read_file(file):
    with open(file, 'r', encoding='utf-8') as f:
        line = f.readlines()
        lines = list(m.replace("\n","") for m in line)
    f.close()
    return lines

In [68]:
save_Content(resultSentence,"ListContent.txt")

In [26]:
# ListContent = read_file("../input/btmidtermnlp/ListContent.txt")

In [69]:
List_1D = resultSentence[0:10000]
# df[8072]
List_1D[0:5]

['mẫu đồng_hồ thông_minh galaxy gear samsung ảnh afp slashgear hôm tương_tác tv thông_minh galaxy gear đảm_nhiệm chức_năng kênh âm_lượng tắt tv galaxy gear khả_năng theo_dõi sức_khỏe tức_là thông_tin sức_khỏe của đeo thiết_bị phát tv nắm được sức_khỏe của được hiện galaxy gear có_thể làm_việc các thiết_bị di_động của samsung như galaxy note galaxy tab inch các thiết_bị như galaxy galaxy note sẽ có cập_nhật để tương_tác được với galaxy gear thành luân',
 'phó_tổng_thống mỹ công_du châu_bàn biển đông nhật ký thỏa_thuận quân_sự philippines thủ_tướng nhật shinzo abe dự_kiến hội_đàm hai lãnh_đạo tái cam_kết_thúc_đẩy hợp_tác song_phương tokyowashington kyodo abe sẽ trao_đổi với phó_tổng_thống mỹ về lập_trường của nhật bản tranh_chấp biển đông biển hoa đông chuyến công_du malaysia singapore và philippines của ông abe tăng_cường hợp_tác kinh_tế và an_ninh với các thành_viên của hiệp_hội các quốc_gia đông_nam asean diễn trong bối_cảnh yêu_sách chủ_quyền cứng_rắn của trung_quốc trên biển đông dấ

# cosine_similarity ------------ Bag of words

In [56]:
# cosine_similarity and bag of word 
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(List_1D)
X = "Bóng đá Việt Nam"
new_vector = vectorizer.transform([X])
similarity_scores = cosine_similarity(new_vector, vectors)
closest_article_idx = np.argmax(similarity_scores)
closest_article = df[closest_article_idx]
print("The closest article to the new article is:")
print(closest_article)

The closest article to the new article is:
MC Thành Trung đã gửi lời chúc mừng đến U23 Việt Nam và HLV Park Hang Seo: "Chúc mừng U23 Việt Nam chúc mừng cột mốc đáng nhớ đầy thuyết phục tiếp theo của bóng đá Việt Nam. Ngoài sự xuất sắc của ông Park thì chưa bao giờ Việt Nam có 1 thủ môn nào chắc chắn, bản lĩnh đến như vậy trong những trận đấu lớn... tuyệt vời Bùi Tiến Dũng".
Thầy trò HLV Park Hang Seo lại vừa viết thêm một trang sử vàng cho nền bóng đá Việt Nam. Với chiến thắng 1-0 trước U23 Syria. Cùng hòa vào niềm vui chung, những nghệ sỹ cũng không kiềm chế nổi cảm xúc, tình yêu mãnh liệt của mình dành cho các cầu thủ U23 - những "người hùng".
MC Nguyên Khang hào hứng chia sẻ trước khi xuống đường ăn mừng: "Chúc mừng đội tuyển Việt Nam. Các bạn rất quả cảm, mạnh mẽ và kiên cường. Cám ơn ông Park và các cầu thủ. Tối nay Việt Nam bão lớn rồi, nếu vào chung kết luôn là tuyệt vời... Đêm nay, em sẽ xuống đường ăn mừng nhá".
NSƯT Chí Trung không kìm được sự phấn khích: "Sướng, sướng quá Vi

# euclidean_distances

In [64]:
distance_scores = euclidean_distances(new_vector, vectors)
# Find the closest article to the new article
closest_article_idx = np.argmin(distance_scores)
closest_article = df[closest_article_idx]
# print("The closest article to the new article is:")
print(closest_article) #7789

Theo.



# jaccard distance

In [58]:
vectorizer = CountVectorizer(binary=True)
vectors = vectorizer.fit_transform(List_1D).toarray()
new_vector = vectorizer.transform([X]).toarray()
similarity_scores = 1 - pairwise_distances(new_vector, vectors, metric='jaccard')
closest_article_idx = np.argmax(similarity_scores)
closest_article = df[closest_article_idx]
print("The closest article to the new article is:")
print(closest_article)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


The closest article to the new article is:
>> >> Xem Ngô Tôn, Han Geng 'đấu đá' trong 'My Kingdom'.
Hồng Giang.
Theo Bưu Điện Việt Nam.

